In [1]:
from zipfile import ZipFile
from ultralytics import YOLO
from p2m.mei2abc import MEIConverter

In [2]:
model = YOLO("../best.pt")

In [3]:
results = model("../resources/samples/000051652-1_2_1.png")


image 1/1 /home/mathieujayet/code/matjayet/PicToMusic/notebooks/../resources/samples/000051652-1_2_1.png: 96x640 1 clef, 1 gamme, 1 metrics, 8 notes, 4 pauses, 427.3ms
Speed: 1.4ms preprocess, 427.3ms inference, 1.8ms postprocess per image at shape (1, 3, 96, 640)


In [4]:
for result in results:
    xywh = result.boxes.xywh  # center-x, center-y, width, height
    xywhn = result.boxes.xywhn  # normalized
    xyxy = result.boxes.xyxy  # top-left-x, top-left-y, bottom-right-x, bottom-right-y
    xyxyn = result.boxes.xyxyn  # normalized
    names = [result.names[cls.item()] for cls in result.boxes.cls.int()]  # class name of each box
    confs = result.boxes.conf  # confidence score of each box

In [5]:
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen


In [6]:
print(xywh.shape)

torch.Size([15, 4])


In [7]:
test_mei = MEIConverter("../resources/samples/000051652-1_2_1.mei")

In [8]:
print(test_mei.mei_to_abc())

X:1
M:2/4
K:Eb
L:1/16
K: clef=C1
Z23 |z4 z2 b,2 |b,6 g,2 |e6 d2 |c2 c2 z4 |]


In [14]:
import numpy as np
un = np.unique_counts(names)
un.values


array(['clef', 'gamme', 'metrics', 'note', 'pause'], dtype='<U7')

In [10]:
print(names)

['pause', 'note', 'note', 'note', 'note', 'note', 'note', 'note', 'clef', 'pause', 'metrics', 'pause', 'pause', 'gamme', 'note']


In [ ]:
def compare_mei_to_parser(file_name : str):
    mei = MEIConverter(file_name +'.mei')
    mei_abc = mei.mei_to_abc()
    mei_key, mei_gamme = mei.score_def["key"], mei.score_def["clef"]
    mei_metrics = None if mei.score_def['meter_count'] == '' else str(mei.score_def['meter_count']) + '/' + str(mei.score_def['meter_unit'])

    print(mei.notes_count, mei.pause_count, mei.score_def)

    result = model(file_name +'.png')[0]
    names = [result.names[cls.item()] for cls in result.boxes.cls.int()]
    unique_names, counts = np.unique(names, return_counts=True)

    # Convertir en dictionnaire pour un accès plus simple
    count_names = dict(zip(unique_names, counts))


    assert count_names['note'] == mei.notes_count
    assert count_names['pause'] == mei.pause_count

    # Accéder au nombre d'occurrences de "note"
    xywhn = result.boxes.xywhn  # normalized

In [44]:
compare_mei_to_parser("../resources/samples/000051652-1_2_1")

7 4 {'key': 'Eb', 'meter_count': 2, 'meter_unit': 4, 'clef': 'C1'}

image 1/1 /home/mathieujayet/code/matjayet/PicToMusic/notebooks/../resources/samples/000051652-1_2_1.png: 96x640 1 clef, 1 gamme, 1 metrics, 8 notes, 4 pauses, 544.3ms
Speed: 3.4ms preprocess, 544.3ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 640)
0


AssertionError: 